In [1]:
a = [j-1 if j < nLat/2+1 else nLat-j+1 for j in range(1, nLat+1)]
window = [np.exp(- 0.5 * (i*dx/sigma)**2) / np.sqrt(2*np.pi) / sigma for i in a]
filter_gauss = np.fft.rfft(window/sum(window), nLat)

field_spectrum = [paper_norm * ((dk*k)**2 + m2eff)**(-0.25) for k in range(nyq)]
phi_x = np.fft.irfft(field_spectrum, nLat) + phi_initial

phi_k = np.fft.rfft(phi_x, nLat)
pspec = [phi_k[k] * np.conj(phi_k[k]) for k in range(nyq)]

spectral_space_smooth_gaussian_field = [phi_k[k]*filter_gauss[k] for k in range(nyq)]
smooth_pspec = [spectral_space_smooth_gaussian_field[k] * np.conj(spectral_space_smooth_gaussian_field[k]) for k in range(nyq)]

def omega_k(k):
    return ((dk*k)**2 + m2eff)**(0.5)

#Taking Fourier transform of the theoretical power spctrum wrt space
def spacelike_correlator(R):
    return sum([pspec[k].real * np.cos(dk*k*dx*R) for k in range(1, nyq)])
def spacelike_correlator_smooth(R):
    return sum([smooth_pspec[k].real * np.cos(dk*k*dx*R) for k in range(1, nyq)])

#Taking Fourier transform of the theoretical power spectrum wrt time
def timelike_correlator(t):
    return sum([pspec[k].real * np.cos(omega_k(k)*dtout*t) for k in range(1, nyq)])
def timelike_correlator_smooth(t):
    return sum([smooth_pspec[k].real * np.cos(omega_k(k)*dtout*t) for k in range(1, nyq)])

NameError: name 'nLat' is not defined

In [2]:
#Spacelike correlator from simulation real space data '_fields.dat'
def spacelike_product(col, sim, R):
    timeslice = random.randrange(nTime)
    x = random.randrange(nLat)
    field = all_real_data[sim][col][timeslice]
    a = x + R
    b = x - R

    if a >= nLat:
        a = a - nLat
    if b < 0:
        b = b + nLat
    return 0.5*(field[x] * field[a] + field[x] * field[b])

#Average over simulations at fixed separation R
def sim_average_of_space_difference_amp(col, R):
    return np.mean([spacelike_product(col, sim, R) for sim in range(nSims)])

#Spacelike correlator from simulation spectral space data (power spectrum) '_fft_fields.dat'
def data_spacelike_correlator(col, sim, R):
    return np.sum([data_pspec(col, sim, k)*np.cos(dk*k*dx*R) for k in range(1, nyq)])

#Average over simulations for fixed separation R
def data_spacelike_correlator_sim_averaged(col, R):
    return np.mean([pspec_fft_spacelike(col, sim, deltat) for sim in range(nSims)])

In [3]:
#Timelike correlator from simulation real space data '_fields.dat'
def timelike_product(col, sim, deltat):
    spaceslice = random.randrange(nLat)
    T = random.randrange(nTime-deltat)
    time_field = [all_real_data[sim][col][i][spaceslice] for i in range(nTime)]
    a = T + deltat
    b = T - deltat

    if a < nTime and b >= 0:
        x = 0.5 * (time_field[T] * time_field[a] + time_field[T] * time_field[b])
    elif a < nTime and b < 0:
        x = time_field[T] * time_field[a]
    elif a >= nTime and b >= 0:
        x = time_field[T] * time_field[b]
    return x

#Average over simulations at fixed time separation deltat
def sim_average_of_time_difference_amp(col, deltat):
    return np.mean([timelike_product(col, sim, deltat) for sim in range(nSims)])

#Timelike correlator from simulation spectral space data (power spectrum '_fft_fields.dat'
def data_timelike_correlator(col, sim, deltat):
    return np.sum([data_pspec(col, sim, k) * np.cos(omega_k(k)*deltat*dtout) for k in range(1, nyq)])

#Average over simulations at fixed time separation deltat
def data_timelike_correlator_sim_averaged(col, deltat):
    return np.mean([pspec_fft_timelike(col, sim, deltat) for sim in range(nSims)])

In [ ]:
def check_decay(col, threshold, sim, timeslice):
    masked_field = add_mask(col, threshold, sim, timeslice)
    if np.count_nonzero(masked_field) != 0:
        return False
    else:
        return True

# Power spectrum from simulation
def data_pspec(col, sim, k, threshold):
    """ This is the P(k)=P(-k) from simulation data for k = 0, 1, ..., nLat/2"""
    timeslice = random.randrange(nTime)
    while check_decay(col, threshold, sim, timeslice) == False:
        if timeslice == 0:
            timeslice = random.randrange(nTime)
        else:
            timeslice = random.randrange(timeslice)
    spec_field = all_real_data[sim][col][timeslice] 
    return spec_field[k] * np.conj(spec_field[k]) # normalisation not present in simulation: nLat for the FT convention, and the 2 from the BM

#Average power specturm over simulations
def data_pspec_sim_averaged(col, k, threshold):
    return np.mean([data_pspec(col, sim, k, threshold) for sim in range(nSims)])

def power_spectrum_theoretical_vs_data(threshold):
    plt.figure(figsize=(7, 5))
    plt.plot(dk*np.arange(1, nyq), [data_pspec_sim_averaged(0, k, threshold) for k in range(1, nyq)], 'bo')
    plt.plot(dk*np.arange(1, nyq), [pspec[k] for k in range(1, nyq)], 'r--', label=r'$P(k)$')
    plt.plot(dk*np.arange(1, nyq), [data_pspec_sim_averaged(2, k, threshold) for k in range(1, nyq)], 'go')
    plt.plot(dk*np.arange(1, nyq), [sm_pspec[k] for k in range(1, nyq)], 'r--', label=r'$P_{sm}(k)$')
    plt.grid()
    plt.legend()
    plt.savefig(plots_file + 'power_spectrum_th_vs_data' + plot_sim_suffix+'.png')
    plt.show()
    return